In [1]:
import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/thomasmartin/ana

In [2]:
import tensorflow as tf

In [3]:
WAVENET_SEARCH_SPACE = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : scope.int(hp.quniform('num_blocks', 2, 20, 2)),
        'block_filters' : scope.int(hp.quniform('block_filters', 16, 256, 16)),
        'residual_filters' : scope.int(hp.quniform('residual_filters', 8, 128, 8)),
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
    },
    'loss' : {'categorical_crossentropy': hp.uniform('cat_loss', 0.1, 1.0),
              'ordinal_squared_error': hp.uniform('ord_loss', 0.1, 1.0)}
}


fit_args = {
    'batch_size' : 16,
    'epochs' : 20,
    'class_weighted' : True
}
CHKPT_FNAME = 'temp_best_weights.h5'


def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    p = {**WAVENET_SEARCH_SPACE, **params}
    p['network_args']['num_blocks'] = int(params['network_args']['num_blocks'])
    p['network_args']['block_filters'] = int(params['network_args']['block_filters'])
    p['network_args']['residual_filters'] = int(params['network_args']['residual_filters'])
    return p

In [4]:
def run_experiment(num_train, max_evals=15, search_space=WAVENET_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=["pseudoGR"],
                              pseudoGR_args={'features' : ['mean', 'var'], 
                                             'per_channel' : True}) for s in train_sets[:2]]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        acc_scores, f1_scores, log_losses = [], [], []
        
        for dset in datasets:
            
            model = NetworkModel(dset, model_args=model_config)
            
            chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
            earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)
            fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
            
            print(f'Training with config: {model_config}')
            
            model.fit(dset, **fit_args)
            
            model.network.load_weights(CHKPT_FNAME)
            
            y_proba = model.predict_proba(dset.X_test)
            y_pred = np.argmax(y_proba, -1)
            
            acc_scores.append(accuracy_score(dset.y_test, y_pred))
            
            f1_scores.append(f1_score(dset.y_test, y_pred, average='macro'))
            
            log_losses.append(log_loss(dset.y_test, y_proba, 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return {'loss' : np.mean(log_losses),
                'acc_scores' : acc_scores,
                'f1_scores' : f1_scores,
                'log_losses' : log_losses,
                'status' : hyperopt.STATUS_OK}
    
    
    trials = Trials()
    
    best_params = hyperopt.fmin(
        fn=train_model,
        space=WAVENET_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        trials=trials
    )
    
    return trials, best_params

In [5]:
trails3, best_params3 = run_experiment(3)

Loading Well:  204-24a-7  from  /home/thomasmartin/Dropbox/core_data/facies/train_data


/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-24a-6  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]
Loading Well:  204-20-3  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-20-1Z  from  /home/thomasmartin/Dropbox/core_data/facies/train_data


/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('pseudoGR', (1917, 32, 8))]
Loading Well:  204-19-3A  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8))]
Loading Well:  205-21b-3  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('pseudoGR', (3842, 32, 8))]
Loading Well:  204-19-7  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9807,)), ('top', (9807,

...

 - ETA: 2s - loss: 1.0807 - acc: 0.6878                                          
                                                                                 
29/42 [===================>..........]                                           
 - ETA: 1s - loss: 1.0676 - acc: 0.6904                                          
                                                                                 
30/42 [====================>.........]                                           
 - ETA: 1s - loss: 1.0595 - acc: 0.6906                                          
                                                                                 
31/42 [=====================>........]                                           
 - ETA: 1s - loss: 1.0712 - acc: 0.6894                                          
                                                                                 
32/42 [=====================>........]                                           
 - ETA: 1s - los

17/23 [=====================>........]                                           
 - ETA: 7s - loss: 3.6667 - acc: 0.2391                                          
                                                                                 
18/23 [======================>.......]                                           
 - ETA: 5s - loss: 3.5531 - acc: 0.2485                                          
                                                                                 
19/23 [=======================>......]                                           
 - ETA: 4s - loss: 3.4615 - acc: 0.2635                                          
                                                                                 
20/23 [=========================>....]                                           
 - ETA: 3s - loss: 3.4750 - acc: 0.2505                                          
                                                                                 
21/23 [=========

                                                                                 
 3/23 [==>...........................]                                           
 - ETA: 3s - loss: 1.2074 - acc: 0.6229                                          
                                                                                 
 4/23 [====>.........................]                                           
 - ETA: 2s - loss: 1.2098 - acc: 0.6232                                          
                                                                                 
 5/23 [=====>........................]                                           
 - ETA: 2s - loss: 1.2144 - acc: 0.6277                                          
                                                                                 
 6/23 [======>.......................]                                           
 - ETA: 2s - loss: 1.2127 - acc: 0.6396                                          
                

 - ETA: 1s - loss: 0.8987 - acc: 0.6923                                          
                                                                                 
13/23 [===============>..............]                                           
 - ETA: 1s - loss: 0.9064 - acc: 0.6889                                          
                                                                                 
14/23 [=================>............]                                           
 - ETA: 1s - loss: 0.8888 - acc: 0.6951                                          
                                                                                 
15/23 [==================>...........]                                           
 - ETA: 1s - loss: 0.8987 - acc: 0.6922                                          
                                                                                 
16/23 [===================>..........]                                           
 - ETA: 1s - los

22/23 [===========================>..]                                           
 - ETA: 0s - loss: 0.8468 - acc: 0.7049                                          
                                                                                 
Epoch 00005: val_loss did not improve from 1.37962
                                                                                 
23/23 [==============================]                                           
 - 29s 1s/step - loss: 0.8450 - acc: 0.7042 - val_loss: 2.0541 - val_acc: 0.5114 

100%|██████████| 15/15 [1:55:54<00:00, 463.62s/it, best loss: 1.1840545490356018]


In [6]:
trails3.results

[{'loss': 1.2710328833305546,
  'acc_scores': [0.6319794301482117, 0.5379918588873813],
  'f1_scores': [0.5021418672458828, 0.38845676305615034],
  'log_losses': [1.1243175687182878, 1.4177481979428215],
  'status': 'ok'},
 {'loss': 27.631021118164064,
  'acc_scores': [0.23649058260342412, 0.08818063831042416],
  'f1_scores': [0.07650380388841928, 0.03241397069785723],
  'log_losses': [27.631021118164064, 27.631021118164064],
  'status': 'ok'},
 {'loss': 1.2982097032982725,
  'acc_scores': [0.6617680874636788, 0.6400802312547932],
  'f1_scores': [0.45845529470443447, 0.46043607169357303],
  'log_losses': [1.2271883519892381, 1.3692310546073065],
  'status': 'ok'},
 {'loss': 1.360151258996948,
  'acc_scores': [0.6353450258168364, 0.5979883192731992],
  'f1_scores': [0.4533829054703652, 0.45246606387563837],
  'log_losses': [1.21363315941273, 1.5066693585811661],
  'status': 'ok'},
 {'loss': 1.277375886378092,
  'acc_scores': [0.6346969918682192, 0.5269010677836116],
  'f1_scores': [0.47

In [7]:
best_params3

{'block_filters': 64.0,
 'cat_loss': 0.8784190182616344,
 'num_blocks': 8.0,
 'ord_loss': 0.7247062123122355,
 'residual_filters': 72.0}

In [8]:
trails3.best_trial

{'state': 2,
 'tid': 6,
 'spec': None,
 'result': {'loss': 1.1840545490356018,
  'acc_scores': [0.6380625875368439, 0.5858356439148132],
  'f1_scores': [0.5366953843187893, 0.4256885262546765],
  'log_losses': [1.0969133866916132, 1.2711957113795904],
  'status': 'ok'},
 'misc': {'tid': 6,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'block_filters': [6],
   'cat_loss': [6],
   'num_blocks': [6],
   'ord_loss': [6],
   'residual_filters': [6]},
  'vals': {'block_filters': [64.0],
   'cat_loss': [0.8784190182616344],
   'num_blocks': [8.0],
   'ord_loss': [0.7247062123122355],
   'residual_filters': [72.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2020, 2, 5, 23, 14, 27, 278000),
 'refresh_time': datetime.datetime(2020, 2, 5, 23, 21, 0, 934000)}

In [11]:
import gc

In [12]:
gc.collect()

2731

In [13]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    pred_dfs = []
    
    for i, train_set in enumerate(train_sets):
        dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['pseudoGR'], 
                             pseudoGR_args={'features' : ['mean', 'var'], 'per_channel' : True})
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
        
        model.fit(dset, callbacks=[chkpt_callback], verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [15]:
params = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 8,
        'block_filters' : 208,
        'residual_filters' : 40,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam'
    }
}

preds_df_3 = pred_results_df(3, params)
preds_df_3.to_csv('pgr_wavenet_results_train_sets3_test.csv')

Loading Well:  204-24a-7  from  /home/thomasmartin/Dropbox/core_data/facies/train_data


/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('pseudoGR', (19294, 32, 8))]
Loading Well:  204-24a-6  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('pseudoGR', (13006, 32, 8))]
Loading Well:  204-20-3  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('pseudoGR', (10906, 32, 8))]
Loading Well:  204-20-1Z  from  /home/thomasmartin/Dropbox/core_data/facies/train_data


/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:88: RuntimeWarning: Mean of empty slice
  output_features.append(np.nanmean(img, axis=1))
/home/thomasmartin/anaconda3/envs/coremdlr/lib/python3.7/site-packages/coremdlr/datasets/utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  output_features.append(np.nanvar(img, axis=1))


Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('pseudoGR', (1917, 32, 8))]
Loading Well:  204-19-3A  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('pseudoGR', (9590, 32, 8))]
Loading Well:  205-21b-3  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('pseudoGR', (3842, 32, 8))]
Loading Well:  204-19-7  from  /home/thomasmartin/Dropbox/core_data/facies/train_data
Extracted pGR features:  ['Umean', 'Rmean', 'Gmean', 'Bmean', 'Uvar', 'Rvar', 'Gvar', 'Bvar']
Feature shapes:  [('depth', (9807,)), ('top', (9807,

TypeError: fit() got multiple values for keyword argument 'callbacks'

In [ ]:
accuracy_score(preds_df_3.y_pred.values, preds_df_3.y_true.values)

NameError: name 'callbacks' is not defined